##### Stage 1: Pre-processing

In [ ]:
# importing necessary libraries
import pandas as pd
import datetime as dt
import math
from pythermalcomfort.models import at

In [1]:
# reading the original dataset extracted from Kaggle
data_raw = pd.read_csv('../../Data/Raw/original_room_dataset.csv')

# converting the column 'Timestamp' to DateTime format
data_raw['Timestamp'] = pd.to_datetime(data_raw.Timestamp)

# locking the columns to be used and setting the index to be the 'Timestamp' column
data_raw['Date'] = data_raw.loc[:, 'Timestamp']
data_raw = data_raw.set_index('Timestamp')

# generating the new dataset, resampling the data to 6h interval and generating the mean
data_processed = data_raw.resample('6h').mean()

# interpolatting the missing values
data_processed = data_processed.interpolate()

# generating a new column to skip the use of the 'Timestamp' index column
for timestamp in data_processed.index:
    data_processed.at[timestamp, 'Date'] = timestamp

# function to remove invalid values
def remove_empty_observations(x):
    return math.isnan(float(x))

# function to round values
def round_observations(x):
    return float(round(x, 0))

# function to set the 'Season' column value for each observation
# WINTER 12/21/20 - 03/20/21 and SPRING - 03/21/21 - 06/20/21
def set_season(date):
    return 'winter' if date < dt.datetime(2021, 3, 20) else 'spring'

# function to set the 'Wind' column value for each observation
# WINTER 0.15 m/s and SPRING - 0.10 m/s
def set_wind(wind):
    return 0.15 if wind == 'winter' else 0.10

# function from 'pythermalcomfort' library to calculate the thermal comfort value with the given parameters
def set_apparent_temperature(temperature_celsius, relative_humidity, wind):
    return at(tdb=temperature_celsius+2,rh=relative_humidity,v=0.25)

# applying the function 'remove_empty_observations' to check for invalid values on the 'Temperature_Empty' and 'Humidity_Empty' columns
data_processed['Temperature_Empty'] = data_processed['Temperature_Celsius'].apply(remove_empty_observations)
data_processed['Humidity_Empty'] = data_processed['Relative_Humidity'].apply(remove_empty_observations)

# applying the function 'set_season' to set the 'Season' column value
data_processed['Season'] = data_processed['Date'].apply(set_season)

# applying the function 'set_wind' to set the 'Wind' column value
data_processed['Wind'] = data_processed['Season'].apply(set_wind)

# applying the function 'round_observations' to round values on the 'Temperature_Empty' and 'Humidity_Empty' columns
data_processed['Temperature_Celsius'] = data_processed['Temperature_Celsius'].apply(round_observations)
data_processed['Relative_Humidity'] = data_processed['Relative_Humidity'].apply(round_observations)

# applying the function 'set_apparent_temperature' to get the 'Apparent_Temperature' column value based on the given ambiance conditions
data_processed['ApparentTemperature'] = data_processed.apply(lambda x: set_apparent_temperature(x['Temperature_Celsius'], x['Relative_Humidity'], x['Wind']), axis=1)

# dropping the rows with missing values for 'Temperature_Celsius' and 'Relative_Humidity'
data_processed.drop(data_processed[((data_processed['Temperature_Empty'] == True) & (data_processed['Humidity_Empty'] == True))].index,inplace=True)

# dropping the 'Temperature_Empty' and 'Humidity_Empty' booleans columns 
data_processed.drop(['Temperature_Empty', 'Humidity_Empty'], axis=1, inplace=True)

# generating the CSV file with the processed data and saving it
data_processed.to_csv('../../Docs/Assets/room.csv')

# generating the CSV file with the processed data and saving it in the local Processed folder
data_processed.to_csv('../../Data/Processed/room.csv')

In [2]:
# checking the number of observations and if everything is all right
data_processed

,Temperature_Celsius,Relative_Humidity,Date,Season,Wind,ApparentTemperature
Timestamp,,,,,,
2021-01-09 12:00:00,21.0,35.0,2021-01-09 12:00:00,winter,0.15,22.1
2021-01-09 18:00:00,23.0,25.0,2021-01-09 18:00:00,winter,0.15,23.4
2021-01-10 00:00:00,21.0,26.0,2021-01-10 00:00:00,winter,0.15,21.2
2021-01-10 06:00:00,21.0,27.0,2021-01-10 06:00:00,winter,0.15,21.3
2021-01-10 12:00:00,21.0,29.0,2021-01-10 12:00:00,winter,0.15,21.5
...,...,...,...,...,...,...
2021-05-13 12:00:00,25.0,51.0,2021-05-13 12:00:00,spring,0.10,28.8
2021-05-13 18:00:00,25.0,53.0,2021-05-13 18:00:00,spring,0.10,29.1
2021-05-14 00:00:00,24.0,55.0,2021-05-14 00:00:00,spring,0.10,27.9


In [3]:
# checking the data types of the columns
data_processed.dtypes

Temperature_Celsius           float64
Relative_Humidity             float64
Date                   datetime64[ns]
Season                         object
Wind                          float64
ApparentTemperature           float64
dtype: object